# Zenodo-Toolbox: Create a Record

This notebook demonstrates how to use the Zenodo Toolbox to create a new record on Zenodo, utilizing both the validator and API functionalities. The process involves several key steps:

1. **Setting Up the Environment**: We'll configure our environment to use either the Zenodo Sandbox (for testing) or the production Zenodo, and set up our API key.

2. **Preparing Metadata**: We'll create a sample metadata dictionary that describes our record. This metadata includes essential information such as the title, description, creators, and license of the dataset.

3. **Validating Metadata**: Before submitting to Zenodo, we'll use the Zenodo Toolbox's validator function to ensure our metadata meets Zenodo's requirements. This step helps catch any errors or missing information before attempting to create the record.

4. **Creating the Record**: Once our metadata is validated, we'll use the Zenodo Toolbox's API functionality to create a new draft record on Zenodo. This step involves sending our metadata to Zenodo's API and handling the response.

5. **Reviewing the Results**: After attempting to create the record, we'll examine the response from Zenodo to confirm whether the record was successfully created and to retrieve important information about the new record.


## Initial Configuration

The first code cell will set up our environment with the following configurations:
- Choose between Zenodo Sandbox (testing) and Production environments.
- Decide whether to use environment variables for API keys or hardcoded keys.
- Set up headers and parameters for API requests.

This setup ensures that we're using the correct environment and API key for our operations. It's crucial to verify these settings before proceeding with creating a record.

Let's begin by running the initial configuration code.


In [ ]:
from datetime import date
import os
from pathlib import Path

os.chdir(Path().absolute().parent) if Path().absolute().name == "Tutorials" else None
from main_functions import create_record
from utilities import append_to_json, printJSON, validate_zenodo_metadata

# Initial Configuration
USE_SANDBOX = True  # Set to True to use the Zenodo Sandbox (testing environment), False for production
ZENODO_BASE_URL = "https://sandbox.zenodo.org" if USE_SANDBOX else "https://zenodo.org"
USE_ENV_API_KEY = True  # Set to False if you want to use hardcoded API keys

# Decide which API key to use based on our configuration
if USE_ENV_API_KEY:
    ZENODO_API_KEY = os.environ.get("ZENODO_SANDBOX_API_KEY") if USE_SANDBOX else os.environ.get("ZENODO_API_KEY")
else:
    ZENODO_API_KEY = "your_sandbox_api_key_here" if USE_SANDBOX else "your_production_api_key_here"

# Set up headers and parameters for our API requests
HEADERS = {"Content-Type": "application/json"}
PARAMS = {"access_token": ZENODO_API_KEY}

print(f"Using {'Sandbox' if USE_SANDBOX else 'Production'} Zenodo Environment.")
print(f"Zenodo API Key is being sourced from {'Environment Variables' if USE_ENV_API_KEY else 'the Code'}")

## Create and Validate Test Metadata

Before creating a new Zenodo record, it's important to prepare and validate the metadata. This step ensures that our data meets Zenodo's requirements before we attempt to create the record.

In the following code cell, we'll create a sample metadata dictionary and then validate it using our `validate_zenodo_metadata` function.

In [ ]:
# Sample metadata for a new Zenodo record.
# -> Try to change any key string to see the errors responses of the validator
test_metadata = {
    "metadata": {
        "title": "Test Dataset for Zenodo API Tutorial",
        "description": "This is a test dataset created using the Zenodo API as part of a beginner's tutorial.",
        "upload_type": "dataset",
        "creators": [{"name": "Doe, John", "affiliation": "Test University"}],
        "access_right": "open",
        "license": "cc-by-4.0",
        "version": "0.0.1",
        "publication_date": date.today().strftime("%Y-%m-%d")
    }
}

print("Validating Metadata...")
validation_errors = validate_zenodo_metadata(test_metadata)

if validation_errors:
    print("\nValidation errors:")
    for error in validation_errors:
        print(f"- {error}")
else:
    print("\nNo validation errors found.")

## Create a New Zenodo Record

Now that we have validated our metadata, we can proceed to create a new Zenodo record. We'll use the `create_record` function to submit our metadata to Zenodo and create a new draft record.

The following code cell attempts to create the record and then displays the results.

In [ ]:
print("Attempting to create a new Zenodo record...")

# Attempt to create a new record
result_msg, result_data = create_record(test_metadata)

# Print the results
print("\nResult Message:")
printJSON(result_msg)
print("\nResult Data:")
printJSON(result_data)

if result_msg["success"]:
    append_to_json(result_data, "Tutorials/Output/sandbox_drafts.json")
    print("\nZenodo Record successfully created! Response Data saved to: Tutorials/Output/sandbox_drafts.json")
else:
    print("\nFailed to create Zenodo Record. Please check the error message above or in result_msg['text'].")


## Interpreting the Results

Let's break down what the results mean:

1. **Success or Failure**: 
   - If `result_msg['success']` is `True`, it means our record was created successfully. 
   - If it's `False`, something went wrong during the creation process.

2. **Response Code**: 
   - `result_msg['response']` gives us the HTTP status code from the API. 
   - A code of 201 means the record was created successfully.
   - Other codes (like 400, 403, etc.) indicate various types of errors.

3. **Message**: 
   - `result_msg['text']` provides a human-readable description of what happened.

4. **Errors**: 
   - If there were any errors, `result_msg['errors']` will contain more detailed information about what went wrong.

5. **Result Data**: 
   - If the record was created successfully, `result_data` will contain information about the new record, such as its ID, URL, and other details.
   - If there was an error, `result_data` might contain additional error information.


You are now able to see the complete response in the file [sandbox_drafts.json](Output/sandbox_drafts.json), which will be used in the next notebooks.

## Preview the Draft

If you are logged in and head to [your Sandbox uploads](https://sandbox.zenodo.org/me/uploads), you should be able to see your recently created draft and preview it as a Zenodo record page.

## Get Information from Draft Data

After successfully creating a draft on Zenodo, we can inspect the data returned in the response. This information is crucial for further operations on the draft, such as updating metadata, uploading files and publishing the upload.

The following code extracts key pieces of information from the `result_data` dictionary, which is assumed to contain the response from Zenodo after creating a draft.

In [ ]:

title = result_data["title"]
concept_record_id = result_data["conceptrecid"]
record_id = result_data["id"]
reserved_doi = result_data["metadata"]["prereserve_doi"]["doi"]

html_link = result_data["links"]["html"]
discard_link = result_data["links"]["discard"]
publish_link = result_data["links"]["publish"]

# Print extracted information
print(f"Title: {title}")
print(f"Concept Record ID: {concept_record_id}")
print(f"Record ID: {record_id}")
print(f"Reserved DOI: {reserved_doi}")
print(f"Draft HTML Link: {html_link}")
print(f"Discard Link: {discard_link}")
print(f"Publish Link: {publish_link}")

### What to do next?

- If the record was created successfully, you have actually created a record draft that remains unpublished. Use the information in `result_data` to access or further operate on this Zenodo record draft.
- Before publishing, we will have to upload files into the deposition, as explained in the next notebook.
- If there was an error, review the error messages to understand what went wrong. Common issues include incorrect API keys, invalid metadata format, or network problems. Rarely — but still possible — internal errors may appear from Zenodo's site, e.g. during maintenances or outages.

Remember, when using the sandbox environment (`USE_SANDBOX = True`), you're working with a test version of Zenodo. This is great for learning and testing, but records created here won't appear on the main Zenodo site and the given DOIs will not be resolved.